<a href="https://colab.research.google.com/github/agilfahrezy11/ee_web_mapping/blob/main/notebooks/implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%bash
cd /content/drive/MyDrive/
mkdir -p /content/drive/MyDrive
git clone https://github.com/agilfahrezy11/ee_web_mapping.git

Cloning into 'ee_web_mapping'...


In [5]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.6 MB/s eta 0:00:00


In [6]:
import sys
import os

# Get the path to the cloned repository
repo_path = '/content/drive/MyDrive/ee_web_mapping'

# Add the repository path to the system path
sys.path.append(repo_path)

# Now you can import files from the repository, for example:
from src.src_modul_1 import Reflectance_Data, Reflectance_Stats

# To verify that the path was added
print(sys.path)

2025-10-22 08:44:11.731 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 08:44:11.733 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-10-22 08:44:11.746 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 08:44:12.157 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-22 08:44:12.158 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 08:44:12.159 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 08:44:12.160 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 08:44:1

['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/ee_web_mapping', '/content/drive/MyDrive/ee_web_mapping', '/usr/local/lib/python3.12/dist-packages/setuptools/_vendor']


In [10]:
import ee
ee.Authenticate()
ee.Initialize(project = "ee-agilakbar")
import geemap
def get_aoi_from_gaul(country="Indonesia", province="Sumatera Selatan"):
    """
    Get Area of Interest geometry from GAUL administrative boundaries.

    Parameters:
    -----------
    country : str
        Country name (default: "Indonesia")
    province : str
        Province/state name (default: "Sumatera Selatan")

    Returns:
    --------
    ee.Geometry : Area of interest geometry
    """
    admin = ee.FeatureCollection("FAO/GAUL/2015/level1")
    aoi_fc = admin.filter(ee.Filter.eq('ADM0_NAME', country)).filter(
        ee.Filter.eq('ADM1_NAME', province)
    )
    return aoi_fc.geometry()

In [12]:
# --- Area of Interest (AOI) ---
# Set the country and province for the AOI using GAUL admin boundaries
COUNTRY = "Indonesia"
PROVINCE = "Sumatera Selatan"
aoi = get_aoi_from_gaul(country=COUNTRY, province=PROVINCE)
#Intialize the relfectance class data function
optical_reflectance = Reflectance_Data()
#define the start and end date for imagery collection
start = '2017-01-01'
end = '2018-12-31'
#get the image collection and corresponding statistics
landsat_data, stats = optical_reflectance.get_optical_data(aoi, start, end, optical_data='L8_SR',
                                                           cloud_cover=40, compute_detailed_stats=False)
#visualization parameter
l8_sr_visparam = {
    'min': 0,
    'max': 0.4,
    'gamma': [0.5, 0.9, 1],
    'bands':['NIR', 'RED', 'GREEN']
}
#create mosaic between image collection, sort them based on the cloud cover property, and clip based on AOI
mosaic_landsat = landsat_data.sort('CLOUD_COVER_LAND').mosaic().clip(aoi)
median_landsat = landsat_data.median().clip(aoi)
#Add the data to the map
Map = geemap.Map()
Map.addLayer(median_landsat, l8_sr_visparam, 'L8 SR Mosaic')
# set center of the map in the area of interest
Map.centerObject(aoi, 7)
# adding the data
Map.addLayer(landsat_data, l8_sr_visparam, 'L8 SR Image Collection')
Map

INFO:Reflectance_Data:ReflectanceData initialized.
INFO:Reflectance_Data:Starting data fetch for Landsat 8 Operational Land Imager Surface Reflectance
INFO:Reflectance_Data:Date range: 2017-01-01 to 2018-12-31
INFO:Reflectance_Data:Cloud cover threshold: 40%
INFO:Reflectance_Data:detailed statistics will not be computed
INFO:Reflectance_Stats:Reflectance Stats initialized.
INFO:Reflectance_Data:Filtered collection created (use compute_detailed_stats=True for more information)


Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…